# YouTube Bot Classifier - Train on GPU

**Google Colab Setup Steps:**
1. **Enable GPU**: Runtime → Change runtime type → Select "T4 GPU" → Save
2. **Run cells sequentially** from top to bottom
3. **Upload dataset.zip** when prompted (345MB - may take 2-3 minutes)
4. **Configure experiment** in the config cell (run single model or compare multiple)
5. **Wait for training** (~15 minutes per model on T4 GPU)
6. **Download results** at the end

**Estimated Time:**
- Single model: ~15-20 minutes
- All 4 models (comparison): ~60-75 minutes

**Free Tier Notes:**
- Colab free tier gives ~12 hours of continuous GPU time
- If disconnected, you'll need to re-upload dataset and restart training

## Quick Start Checklist

Before running, verify:
- [ ] GPU is enabled (Runtime → Change runtime type → GPU)
- [ ] You have dataset.zip ready to upload (345MB)
- [ ] You know which config to run (single model or comparison)

**Notebook Flow:**
1. Check GPU ✓
2. Upload dataset ✓
3. Extract and verify data ✓
4. Configure experiment ✓
5. Train model(s) ✓
6. Review results ✓
7. Download models and exports ✓

In [ ]:
# Check GPU availability
!nvidia-smi

# Verify CUDA is available
import torch
print(f"\n🔍 GPU Check:")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n✅ GPU is ready!")
else:
    print("\n⚠️  WARNING: GPU not detected!")
    print("   Go to: Runtime → Change runtime type → Select GPU")

In [ ]:
# Upload dataset
from google.colab import files
import os

print('📦 Upload dataset.zip (345MB)')
print('⏱️  This may take 2-3 minutes depending on your connection...')
print()

# Check if already uploaded
if os.path.exists('dataset.zip'):
    print('✅ dataset.zip already exists!')
    response = input('Re-upload? (y/n): ')
    if response.lower() != 'y':
        print('Skipping upload, using existing file.')
    else:
        uploaded = files.upload()
else:
    uploaded = files.upload()

print('✅ Upload complete!')

In [ ]:
# Extract dataset
import os

if not os.path.exists('dataset'):
    print('📂 Extracting dataset.zip...')
    !unzip -q dataset.zip
    print('✅ Extraction complete!')
else:
    print('✅ Dataset already extracted!')

print('\n📊 Dataset structure:')
!ls -lh dataset/train dataset/val

# Count images
import os
train_bot = len(os.listdir('dataset/train/bot'))
train_not_bot = len(os.listdir('dataset/train/not_bot'))
val_bot = len(os.listdir('dataset/val/bot'))
val_not_bot = len(os.listdir('dataset/val/not_bot'))

print(f'\n📈 Dataset summary:')
print(f'   Training: {train_bot} bots, {train_not_bot} not_bots (total: {train_bot + train_not_bot})')
print(f'   Validation: {val_bot} bots, {val_not_bot} not_bots (total: {val_bot + val_not_bot})')
print(f'   Class balance: {train_bot/(train_bot + train_not_bot)*100:.1f}% bots in training')

In [ ]:
# Install/Import required libraries
print('📚 Importing libraries...')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import numpy as np
import cv2
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

# Set up device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'✅ Libraries loaded successfully!')
print(f'🎯 Using device: {DEVICE}')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print('🎲 Random seeds set for reproducibility')

## Experiment Configuration

**Easy Mode**: Just change `SELECTED_CONFIG` below to run a single model:
- 0 = MobileNetV2 with equal weights
- 1 = MobileNetV2 with weighted classes
- 2 = ResNet18 with equal weights
- 3 = ResNet18 with weighted classes
- 4 = ResNet34 with equal weights
- 5 = ResNet34 with weighted classes
- 'all' = Run all configurations and compare

**Advanced Mode**: Edit the CONFIGS list to add your own configurations or tweak parameters.

**Model Info:**
- **MobileNetV2**: Lightweight, fast training (~12-15 min/config)
- **ResNet18**: Small ResNet, good balance (~15-18 min/config)
- **ResNet34**: Deeper ResNet, potentially more accurate (~18-22 min/config)

In [ ]:
# ============================================================================
# CONFIGURE YOUR EXPERIMENT HERE
# ============================================================================

# Which config to run? Set to a number (0-5) or 'all'
SELECTED_CONFIG = 0  # Change this to: 0, 1, 2, 3, 4, 5, or 'all'

# Define all available configurations
CONFIGS = [
    # Config 0: MobileNetV2 with equal weights
    {
        'name': 'mobilenet_equal',
        'model': 'mobilenet_v2',
        'class_weight_bot': 1.0,
        'batch_size': 64,
        'learning_rate': 0.001,
        'num_epochs': 15,
    },
    # Config 1: MobileNetV2 with weighted classes (prioritize bot detection)
    {
        'name': 'mobilenet_weighted',
        'model': 'mobilenet_v2',
        'class_weight_bot': 1.65,
        'batch_size': 64,
        'learning_rate': 0.001,
        'num_epochs': 15,
    },
    # Config 2: ResNet18 with equal weights
    {
        'name': 'resnet18_equal',
        'model': 'resnet18',
        'class_weight_bot': 1.0,
        'batch_size': 64,
        'learning_rate': 0.001,
        'num_epochs': 15,
    },
    # Config 3: ResNet18 with weighted classes (prioritize bot detection)
    {
        'name': 'resnet18_weighted',
        'model': 'resnet18',
        'class_weight_bot': 1.65,
        'batch_size': 64,
        'learning_rate': 0.001,
        'num_epochs': 15,
    },
    # Config 4: ResNet34 with equal weights
    {
        'name': 'resnet34_equal',
        'model': 'resnet34',
        'class_weight_bot': 1.0,
        'batch_size': 64,
        'learning_rate': 0.001,
        'num_epochs': 15,
    },
    # Config 5: ResNet34 with weighted classes (prioritize bot detection)
    {
        'name': 'resnet34_weighted',
        'model': 'resnet34',
        'class_weight_bot': 1.65,
        'batch_size': 64,
        'learning_rate': 0.001,
        'num_epochs': 15,
    },
]

# ============================================================================
# END CONFIGURATION
# ============================================================================

# Determine which configs to run
if SELECTED_CONFIG == 'all':
    configs_to_run = CONFIGS
    print(f"🔄 Running ALL {len(CONFIGS)} configurations for comparison:")
else:
    configs_to_run = [CONFIGS[SELECTED_CONFIG]]
    print(f"🎯 Running SINGLE configuration:")

for i, cfg in enumerate(configs_to_run):
    print(f"  - {cfg['name']}: {cfg['model']}, bot_weight={cfg['class_weight_bot']}, epochs={cfg['num_epochs']}")

all_results = []

In [ ]:
class AvatarDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.samples = []
        for label_dir in ['bot', 'not_bot']:
            label = 0 if label_dir == 'bot' else 1
            for img_path in (self.root_dir / label_dir).glob('*.png'):
                self.samples.append((str(img_path), label))
        bot_count = sum(1 for _, l in self.samples if l == 0)
        print(f'{root_dir}: {len(self.samples)} samples (bot={bot_count}, not_bot={len(self.samples)-bot_count})')
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        return self.transform(image) if self.transform else image, label

In [ ]:
train_transform = transforms.Compose([
    transforms.ToPILImage(), transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.ToPILImage(), transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_dataset = AvatarDataset('dataset/train', train_transform)
test_dataset = AvatarDataset('dataset/val', test_transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    loss_sum, correct, total = 0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return loss_sum / len(loader), 100. * correct / total

def test_epoch(model, loader, criterion, device):
    model.eval()
    loss_sum, correct, total = 0, 0, 0
    all_preds, all_labels, all_probs = [], [], []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss_sum += criterion(outputs, labels).item()
            probs = torch.softmax(outputs, 1)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    return (loss_sum / len(loader), 100. * correct / total,
            np.array(all_preds), np.array(all_labels), np.array(all_probs))

In [ ]:
# Train selected configuration(s)
import time

for config_idx, config in enumerate(configs_to_run):
    print("\n" + "="*80)
    print(f"TRAINING CONFIG: {config['name']} ({config_idx+1}/{len(configs_to_run)})")
    print("="*80)
    
    # Extract hyperparameters
    model_name = config['model']
    class_weight_bot = config['class_weight_bot']
    lr = config['learning_rate']
    epochs = config['num_epochs']
    config_name = config['name']
    
    print(f"📋 Configuration:")
    print(f"   Model: {model_name}")
    print(f"   Class weight (bot): {class_weight_bot}")
    print(f"   Learning rate: {lr}")
    print(f"   Epochs: {epochs}")
    print()
    
    # Build model
    if model_name == 'mobilenet_v2':
        model = torchvision.models.mobilenet_v2(pretrained=True)
        model.classifier[1] = nn.Linear(model.last_channel, 2)
    elif model_name == 'resnet18':
        model = torchvision.models.resnet18(pretrained=True)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 2)
    elif model_name == 'resnet34':
        model = torchvision.models.resnet34(pretrained=True)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 2)
    else:
        raise ValueError(f"Unknown model: {model_name}")
    
    model = model.to(DEVICE)
    
    # Setup training
    class_weights = torch.tensor([class_weight_bot, 1.0]).to(DEVICE)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5)
    
    best_recall = 0.0
    history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': [], 'bot_recall': []}
    
    start_time = time.time()
    
    # Training loop
    print("🏋️ Training started...")
    for epoch in range(epochs):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
        test_loss, test_acc, preds, labels, probs = test_epoch(model, test_loader, criterion, DEVICE)
        cm = confusion_matrix(labels, preds)
        bot_recall = cm[0,0] / (cm[0,0] + cm[0,1]) if (cm[0,0] + cm[0,1]) > 0 else 0
        
        print(f'Epoch {epoch+1:2d}/{epochs} | Train: {train_acc:5.2f}% | Test: {test_acc:5.2f}% | Bot Recall: {bot_recall:.4f}', end='')
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        history['bot_recall'].append(bot_recall)
        
        scheduler.step(bot_recall)
        
        if bot_recall > best_recall:
            best_recall = bot_recall
            model_filename = f'{config_name}.pth'
            torch.save(model.state_dict(), model_filename)
            print(' ✓ [BEST]')
        else:
            print()
    
    training_time = time.time() - start_time
    
    # Final evaluation
    print(f"\n📊 Evaluating best model...")
    model.load_state_dict(torch.load(model_filename))
    test_loss, test_acc, preds, labels, probs = test_epoch(model, test_loader, criterion, DEVICE)
    cm = confusion_matrix(labels, preds)
    bot_recall = cm[0,0] / (cm[0,0] + cm[0,1])
    bot_precision = cm[0,0] / (cm[0,0] + cm[1,0])
    bot_f1 = 2 * bot_precision * bot_recall / (bot_precision + bot_recall)
    roc_auc = roc_auc_score(labels, probs[:, 0])
    
    # Print detailed results
    print(f"\n✅ RESULTS for {config_name}:")
    print(f"   Test Accuracy: {test_acc:.2f}%")
    print(f"   Bot Recall: {bot_recall:.4f} (catches {bot_recall*100:.1f}% of bots)")
    print(f"   Bot Precision: {bot_precision:.4f} ({bot_precision*100:.1f}% accurate when predicting bot)")
    print(f"   Bot F1 Score: {bot_f1:.4f}")
    print(f"   ROC AUC: {roc_auc:.4f}")
    print(f"   Training time: {training_time/60:.1f} minutes")
    print(f"\n   Confusion Matrix:")
    print(f"   {cm}")
    
    # Store results
    result = {
        'config_name': config_name,
        'model': model_name,
        'class_weight_bot': class_weight_bot,
        'test_accuracy': test_acc,
        'bot_recall': bot_recall,
        'bot_precision': bot_precision,
        'bot_f1': bot_f1,
        'roc_auc': roc_auc,
        'training_time': training_time,
        'confusion_matrix': cm,
        'history': history,
    }
    all_results.append(result)

print("\n" + "="*80)
if len(configs_to_run) == 1:
    print("✅ TRAINING COMPLETE!")
else:
    print(f"✅ ALL {len(configs_to_run)} TRAININGS COMPLETE!")
print("="*80)

In [ ]:
# Results summary and comparison
import pandas as pd

if len(all_results) > 1:
    # COMPARISON MODE - Multiple configs were run
    print("\n" + "="*80)
    print("HYPERPARAMETER TUNING RESULTS - COMPARISON")
    print("="*80)
    
    # Create comparison table
    comparison_data = []
    for result in all_results:
        comparison_data.append({
            'Config': result['config_name'],
            'Model': result['model'],
            'Bot Weight': result['class_weight_bot'],
            'Test Acc (%)': f"{result['test_accuracy']:.2f}",
            'Bot Recall': f"{result['bot_recall']:.4f}",
            'Bot Precision': f"{result['bot_precision']:.4f}",
            'Bot F1': f"{result['bot_f1']:.4f}",
            'ROC AUC': f"{result['roc_auc']:.4f}",
            'Time (min)': f"{result['training_time']/60:.1f}",
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print("\n")
    print(comparison_df.to_string(index=False))
    
    # Find best configuration for bot recall
    best_idx = max(range(len(all_results)), key=lambda i: all_results[i]['bot_recall'])
    best_result = all_results[best_idx]
    
    print("\n" + "="*80)
    print("🏆 BEST CONFIGURATION (by Bot Recall)")
    print("="*80)
    print(f"Config: {best_result['config_name']}")
    print(f"Model: {best_result['model']}")
    print(f"Bot Class Weight: {best_result['class_weight_bot']}")
    print(f"Bot Recall: {best_result['bot_recall']:.4f}")
    print(f"Bot Precision: {best_result['bot_precision']:.4f}")
    print(f"Bot F1 Score: {best_result['bot_f1']:.4f}")
    print(f"ROC AUC: {best_result['roc_auc']:.4f}")
    print(f"Test Accuracy: {best_result['test_accuracy']:.2f}%")
    print(f"\nConfusion Matrix:")
    print(best_result['confusion_matrix'])
    print("="*80)
else:
    # SINGLE MODE - Only one config was run
    result = all_results[0]
    print("\n" + "="*80)
    print(f"FINAL RESULTS: {result['config_name']}")
    print("="*80)
    print(f"Model: {result['model']}")
    print(f"Bot Class Weight: {result['class_weight_bot']}")
    print(f"\n📈 Metrics:")
    print(f"  Test Accuracy: {result['test_accuracy']:.2f}%")
    print(f"  Bot Recall: {result['bot_recall']:.4f}")
    print(f"  Bot Precision: {result['bot_precision']:.4f}")
    print(f"  Bot F1 Score: {result['bot_f1']:.4f}")
    print(f"  ROC AUC: {result['roc_auc']:.4f}")
    print(f"  Training Time: {result['training_time']/60:.1f} minutes")
    print(f"\nConfusion Matrix:")
    print(result['confusion_matrix'])
    print("="*80)
    
    best_result = result  # For use in later cells

In [ ]:
# Visualizations
if len(all_results) > 1:
    # COMPARISON MODE - Multiple configs
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Bot Recall comparison (bar chart)
    config_names = [r['config_name'] for r in all_results]
    bot_recalls = [r['bot_recall'] for r in all_results]
    
    # Color by model type
    colors = []
    for name in config_names:
        if 'mobilenet' in name:
            colors.append('blue')
        elif 'resnet34' in name:
            colors.append('red')
        elif 'resnet18' in name:
            colors.append('green')
        else:
            colors.append('gray')
    
    axes[0, 0].bar(range(len(config_names)), bot_recalls, color=colors, alpha=0.7)
    axes[0, 0].set_xticks(range(len(config_names)))
    axes[0, 0].set_xticklabels(config_names, rotation=45, ha='right', fontsize=8)
    axes[0, 0].set_ylabel('Bot Recall')
    axes[0, 0].set_title('Bot Recall Comparison')
    axes[0, 0].grid(True, axis='y')
    axes[0, 0].axhline(y=max(bot_recalls), color='r', linestyle='--', alpha=0.5)
    
    # Plot 2: F1 Score comparison
    bot_f1s = [r['bot_f1'] for r in all_results]
    axes[0, 1].bar(range(len(config_names)), bot_f1s, color=colors, alpha=0.7)
    axes[0, 1].set_xticks(range(len(config_names)))
    axes[0, 1].set_xticklabels(config_names, rotation=45, ha='right', fontsize=8)
    axes[0, 1].set_ylabel('Bot F1 Score')
    axes[0, 1].set_title('Bot F1 Score Comparison')
    axes[0, 1].grid(True, axis='y')
    
    # Plot 3: Recall vs Precision scatter
    bot_precisions = [r['bot_precision'] for r in all_results]
    for i, (recall, precision, name) in enumerate(zip(bot_recalls, bot_precisions, config_names)):
        axes[1, 0].scatter(recall, precision, s=200, alpha=0.7, color=colors[i], label=name)
    axes[1, 0].set_xlabel('Bot Recall')
    axes[1, 0].set_ylabel('Bot Precision')
    axes[1, 0].set_title('Recall vs Precision Trade-off')
    axes[1, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=7)
    axes[1, 0].grid(True)
    
    # Plot 4: Training curves for best model
    best_history = best_result['history']
    ax1 = axes[1, 1]
    ax1.plot(best_history['bot_recall'], marker='o', color='green', linewidth=2, label='Bot Recall')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Bot Recall', color='green')
    ax1.tick_params(axis='y', labelcolor='green')
    ax1.set_title(f'Best Model Training: {best_result["config_name"]}')
    ax1.grid(True)
    
    ax2 = ax1.twinx()
    ax2.plot(best_history['test_acc'], marker='s', color='blue', linewidth=2, label='Test Accuracy')
    ax2.set_ylabel('Test Accuracy (%)', color='blue')
    ax2.tick_params(axis='y', labelcolor='blue')
    
    plt.tight_layout()
    plt.savefig('hyperparameter_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("📊 Comparison plots saved to hyperparameter_comparison.png")

else:
    # SINGLE MODE - Training curves for single config
    result = all_results[0]
    history = result['history']
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Plot 1: Loss
    axes[0].plot(history['train_loss'], marker='o', label='Train', linewidth=2)
    axes[0].plot(history['test_loss'], marker='s', label='Test', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Loss over Epochs')
    axes[0].legend()
    axes[0].grid(True)
    
    # Plot 2: Accuracy
    axes[1].plot(history['train_acc'], marker='o', label='Train', linewidth=2)
    axes[1].plot(history['test_acc'], marker='s', label='Test', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title('Accuracy over Epochs')
    axes[1].legend()
    axes[1].grid(True)
    
    # Plot 3: Bot Recall
    axes[2].plot(history['bot_recall'], marker='o', color='green', linewidth=2)
    axes[2].axhline(y=result['bot_recall'], color='r', linestyle='--', 
                    label=f'Best: {result["bot_recall"]:.4f}')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Bot Recall')
    axes[2].set_title('Bot Recall over Epochs')
    axes[2].legend()
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.savefig(f'training_{result["config_name"]}.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"📊 Training plots saved to training_{result['config_name']}.png")

In [ ]:
# Download results
from google.colab import files

if len(all_results) > 1:
    # Download best model from comparison
    best_model_filename = f"{best_result['config_name']}.pth"
    print(f"📥 Downloading best model: {best_model_filename}")
    files.download(best_model_filename)
    files.download('hyperparameter_comparison.png')
    
    print(f"\n✅ Downloaded:")
    print(f"   - {best_model_filename} (best performing model)")
    print(f"   - hyperparameter_comparison.png (visual comparison)")
    print(f"\n🏆 Best config was: {best_result['config_name']}")
    print(f"   Upload {best_model_filename} to your VPS for inference.")
else:
    # Download single model
    result = all_results[0]
    model_filename = f"{result['config_name']}.pth"
    plot_filename = f"training_{result['config_name']}.png"
    
    print(f"📥 Downloading model and plots...")
    files.download(model_filename)
    files.download(plot_filename)
    
    print(f"\n✅ Downloaded:")
    print(f"   - {model_filename} (trained model)")
    print(f"   - {plot_filename} (training curves)")
    print(f"\n📝 Config: {result['config_name']}")
    print(f"   Upload {model_filename} to your VPS for inference.")

In [ ]:
# Export results to files
import json
import pandas as pd
from datetime import datetime

# Create timestamp for this experiment
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

if len(all_results) > 1:
    # COMPARISON MODE - Export comparison table
    
    # 1. Export to CSV (easy to open in Excel/Google Sheets)
    csv_filename = f'results_comparison_{timestamp}.csv'
    comparison_data = []
    for result in all_results:
        comparison_data.append({
            'config_name': result['config_name'],
            'model': result['model'],
            'class_weight_bot': result['class_weight_bot'],
            'test_accuracy': result['test_accuracy'],
            'bot_recall': result['bot_recall'],
            'bot_precision': result['bot_precision'],
            'bot_f1': result['bot_f1'],
            'roc_auc': result['roc_auc'],
            'training_time_minutes': result['training_time'] / 60,
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df.to_csv(csv_filename, index=False)
    print(f"📊 Exported comparison to: {csv_filename}")
    
    # 2. Export to JSON (includes full details like confusion matrices)
    json_filename = f'results_full_{timestamp}.json'
    export_data = {
        'timestamp': timestamp,
        'num_configs': len(all_results),
        'best_config': best_result['config_name'],
        'results': []
    }
    
    for result in all_results:
        export_data['results'].append({
            'config_name': result['config_name'],
            'model': result['model'],
            'class_weight_bot': result['class_weight_bot'],
            'metrics': {
                'test_accuracy': float(result['test_accuracy']),
                'bot_recall': float(result['bot_recall']),
                'bot_precision': float(result['bot_precision']),
                'bot_f1': float(result['bot_f1']),
                'roc_auc': float(result['roc_auc']),
                'training_time_minutes': float(result['training_time'] / 60),
            },
            'confusion_matrix': result['confusion_matrix'].tolist(),
            'history': {
                'train_loss': [float(x) for x in result['history']['train_loss']],
                'train_acc': [float(x) for x in result['history']['train_acc']],
                'test_loss': [float(x) for x in result['history']['test_loss']],
                'test_acc': [float(x) for x in result['history']['test_acc']],
                'bot_recall': [float(x) for x in result['history']['bot_recall']],
            }
        })
    
    with open(json_filename, 'w') as f:
        json.dump(export_data, f, indent=2)
    print(f"📋 Exported full results to: {json_filename}")
    
    print(f"\n💾 Files created:")
    print(f"   - {csv_filename} (summary table for Excel/Sheets)")
    print(f"   - {json_filename} (full details including training history)")
    
else:
    # SINGLE MODE - Export single config results
    
    result = all_results[0]
    
    # 1. Export to CSV
    csv_filename = f'results_{result["config_name"]}_{timestamp}.csv'
    result_data = pd.DataFrame([{
        'config_name': result['config_name'],
        'model': result['model'],
        'class_weight_bot': result['class_weight_bot'],
        'test_accuracy': result['test_accuracy'],
        'bot_recall': result['bot_recall'],
        'bot_precision': result['bot_precision'],
        'bot_f1': result['bot_f1'],
        'roc_auc': result['roc_auc'],
        'training_time_minutes': result['training_time'] / 60,
    }])
    result_data.to_csv(csv_filename, index=False)
    print(f"📊 Exported results to: {csv_filename}")
    
    # 2. Export to JSON (full details)
    json_filename = f'results_{result["config_name"]}_{timestamp}.json'
    export_data = {
        'timestamp': timestamp,
        'config_name': result['config_name'],
        'model': result['model'],
        'class_weight_bot': result['class_weight_bot'],
        'metrics': {
            'test_accuracy': float(result['test_accuracy']),
            'bot_recall': float(result['bot_recall']),
            'bot_precision': float(result['bot_precision']),
            'bot_f1': float(result['bot_f1']),
            'roc_auc': float(result['roc_auc']),
            'training_time_minutes': float(result['training_time'] / 60),
        },
        'confusion_matrix': result['confusion_matrix'].tolist(),
        'history': {
            'train_loss': [float(x) for x in result['history']['train_loss']],
            'train_acc': [float(x) for x in result['history']['train_acc']],
            'test_loss': [float(x) for x in result['history']['test_loss']],
            'test_acc': [float(x) for x in result['history']['test_acc']],
            'bot_recall': [float(x) for x in result['history']['bot_recall']],
        }
    }
    
    with open(json_filename, 'w') as f:
        json.dump(export_data, f, indent=2)
    print(f"📋 Exported full results to: {json_filename}")
    
    print(f"\n💾 Files created:")
    print(f"   - {csv_filename} (metrics summary)")
    print(f"   - {json_filename} (full details including training history)")

In [ ]:
# Download exported files (optional - downloads CSV and JSON)
from google.colab import files

# List of files to download
export_files = []

if len(all_results) > 1:
    # Comparison mode - find the files we just created
    import glob
    export_files.extend(glob.glob('results_comparison_*.csv'))
    export_files.extend(glob.glob('results_full_*.json'))
else:
    # Single mode
    import glob
    export_files.extend(glob.glob(f'results_{all_results[0]["config_name"]}_*.csv'))
    export_files.extend(glob.glob(f'results_{all_results[0]["config_name"]}_*.json'))

print("📥 Downloading exported result files...")
for filename in export_files:
    print(f"   Downloading: {filename}")
    files.download(filename)

print(f"\n✅ Downloaded {len(export_files)} result file(s)")
print("\n💡 Tip: Keep these files to track your experiments over time!")

## 🎉 Training Complete!

**What to do next:**

1. **Review results** in the cells above (metrics table and visualizations)
2. **Download your model** - The best performing model will be downloaded automatically
3. **Export results** - CSV and JSON files contain all experiment details
4. **Save to Google Drive** (optional) - See cell below to backup everything

**Using the trained model:**
- Upload the `.pth` file to your VPS
- Load it with: `model.load_state_dict(torch.load('model_name.pth'))`
- Use for inference on new avatar images

In [ ]:
# (Optional) Save all results to Google Drive
# Uncomment and run this cell to backup everything to your Google Drive

"""
from google.colab import drive
import shutil
import os

# Mount Google Drive
print('📁 Mounting Google Drive...')
drive.mount('/content/drive')

# Create backup folder
backup_folder = '/content/drive/MyDrive/avatar_classifier_results'
os.makedirs(backup_folder, exist_ok=True)

# Copy all result files
print('💾 Backing up to Google Drive...')
files_to_backup = []

# Model files
import glob
files_to_backup.extend(glob.glob('*.pth'))
files_to_backup.extend(glob.glob('*.png'))
files_to_backup.extend(glob.glob('*.csv'))
files_to_backup.extend(glob.glob('*.json'))

for file in files_to_backup:
    if os.path.exists(file):
        shutil.copy(file, backup_folder)
        print(f'   ✓ Backed up: {file}')

print(f'\n✅ All files backed up to: {backup_folder}')
print('💡 Your results are now safely stored in Google Drive!')
"""

print('ℹ️  Uncomment the code above to backup results to Google Drive')
print('   This is useful if you want to save everything permanently.')